# Simple ETL Example
This example demonstrates a basic Extract, Transform, Load pattern for taking measurement data in various formats and normalizing to SystemLink DataFrames. It utilizes the ***File Service*** to download a specified file, the ***DataFrame Service*** to store the normalized test and measurement data, and the ***Test Monitor Service*** to link the metadata from the test data to the created DataTable in a highly searchable manner.

### Imports
Import Python modules for executing the notebook. The requests and json libraries are used for communicating with the various SystemLink service's enpoints. Pandas is used for building and handling dataframe. Scrapbook is used for running notebooks and recording data for the SystemLink Notebook Execution Service.

The SYSTEMLINK_API_KEY environment variable specifies an API key created for the user executinge this notebook, which provides Role Based Access Control to the various SystemLink APIs called by this notebook. The API key will expire after 24 hours.  
The SYSTEMLINK_HTTP_URI environment variable gives the base URL to the SystemLink instance executing this notebook.

In [ ]:
import os
import requests
import json
import pandas
import scrapbook

api_key = os.getenv("SYSTEMLINK_API_KEY")
sl_uri = os.getenv("SYSTEMLINK_HTTP_URI")

### Parameters
These are the parameters that the notebook expects to be passed in by SystemLink. For notebooks designed to be triggered by file actions as defined in the Routines Service or by pressing the 'Analyze' button in one of the SystemLink files grids, they must tag the cell with 'parameters' and at minimum specify the following in the cell metadata using the JupyterLab Property Inspector (double gear icon):
```    
{
    "papermill": {
        "parameters": {
            "file_ids": []
        }
    },
    "tags": [
        "parameters"
    ],
    "systemlink": {
        "interfaces": [
            "ni-files"
        ],
        "parameters": [
            {
                "display_name": "file_ids",
                "id": "file_ids",
                "type": "string[]"
            }
        ],
        "version": 2
    }
}
```

For more information on how paramaterization works, review the [papermill documentation](https://papermill.readthedocs.io/en/latest/usage-parameterize.html#how-parameters-work).

In [ ]:
file_ids = ["2a57355d-1e1e-4ca3-be79-718fee9372b7"]

### Download the file specified by the input parameters and store it locally

In [ ]:
# This notebook is written to operate on 1 file. Throw a read-able exception if the expected parameters are not present.
if len(file_ids) != 1:
    raise Exception(f"This example notebook is designed to operate on one file at a time.")
    
file_id = file_ids[0]

headers = { 'X-NI-API-KEY': api_key }
download_resp = requests.get(f'{sl_uri}/nifile/v1/service-groups/Default/files/{file_id}/data', headers=headers)

# Expect a 200 code on success, raise an exception if there is an error response
download_resp.raise_for_status()

# The name of the file is returned in the content-disposition section of the header in the format:
#    attachment; filename="MeasurementData.csv"
# This returns everything between the first and last " characters to use as the local filename.
filename = download_resp.headers['content-disposition'].split('"')[1::-1][0]

with open(filename, 'wb') as file:
    file.write(download_resp.content)

### Select the Workspace to create the test result and data table
SystemLink stores all artifacts and data in Workspaces. These Workspaces provide a key component of our role-based access control mechanism. Each user may have one or more Roles in a given Workspace that grant them various privileges for SystemLink's services. For example, the ability to create a new test result.

Here you should ensure you select a workspace that you have sufficient privileges to create and update both test results and datatables.

In [ ]:
headers = { 'X-NI-API-KEY': api_key }
get_workspaces_resp = requests.get(f'{sl_uri}/niuser/v1/workspaces', headers=headers)

# Expect a 200 code on success, raise an exception if there is an error response
get_workspaces_resp.raise_for_status()

# Print the list of workspaces you may choose to create the test result and datatable in
workspaces = get_workspaces_resp.json()["workspaces"]
for workspace in workspaces:
    if workspace["enabled"]: print(workspace["name"])

# Select the desired workspace. As a placeholder, I'm simply selecting the first in the list here
workspace = workspaces[0]

### Load the test data in the provided file into pandas for analysis
Depending on your data's format, additional processing may need to be done here to extract test metadata from the file. In this example, the specified data format is a simple CSV file with 5 columns of various data types.

note: Keep in mind that the images running automated executions are limited to 1 GB of RAM. Loading large files and operating on them in memory is not recommended.

In [ ]:
df = pandas.read_csv(filename, sep=',', index_col=0)

### Create a new Table that matches the data format of the specified file
In this example, we create a new DataTable for each file. You may also use the SystemLink DataFrame Service to query for existing tables if you would like to append multiple files into the same table.

In [ ]:
# Specify the name and datatype for the columns to create in the DataTable
columnNames = list(df)
columnTypes = ['TIMESTAMP', 'FLOAT32', 'FLOAT32', 'STRING', 'BOOL']

# Create column specification (to pass to the DataFrame service route to create a table)
# Includes column name, column data type, and column type. The first column is set as the index column.
columns = []
first_column = True
for (columnName, dataType) in zip(columnNames, columnTypes):
    columnType = 'INDEX' if first_column else 'NULLABLE'
    first_column = False
    columns.append({'name': columnName, 'dataType': dataType, 'columnType' : columnType})

# Create the table
table_info = {'name': filename, 'columns': columns, 'workspace': workspace["id"]}
headers = { "X-NI-API-KEY": api_key, 'Content-Type': 'application/json' }
create_table_resp = requests.post(f'{sl_uri}/nidataframe/v1/tables', headers=headers, json=table_info)

# Expect 201 on success, raise an exception if there is an error response
create_table_resp.raise_for_status()

table_id = create_table_resp.json()["id"]

### Write information from the pandas dataframe into the SystemLink DataFrame Service
Use the Dataframe service to write the data from the measurement file into SystemLink. In this scenario, we are writing the entire file to a table at once. For data sets larger than 256 MB, multiple writes will be required.

In [ ]:
# Convert the pandas dataframe to the expected json format
frame_dict = df.astype(str).to_json(orient="split", index=False)

# Write the data to the table. Since we're writing one file/frame per table, set endofData to true
frame_info = f'{{"frame": {frame_dict}, "endOfData": true}}'
headers = {'X-NI-API-KEY': api_key, 'Content-Type': 'application/json'}
write_data_resp = requests.post(f'{sl_uri}/nidataframe/v1/tables/{table_id}/data', headers=headers, data=frame_info)

# Expect 204 on success, raise an exception if there is an error response
write_data_resp.raise_for_status()

### Create new Test Result
Use the TestMonitor API to create a new Test Result. In a real application, the result data will likely be provided by metadata in the file.

In [ ]:
# Form the JSON request body as per the nitestmonitor/v2 API
results_info = {
    "results": [
        {
            "programName": "Simple ETL Example",
            "status": {
                "statusType": "PASSED",
                "statusName": "Passed"
            },
            "partNumber": "my-product",
            "systemId": "my-system",
            "hostName": "My-Host",
            "serialNumber": "123-456",
            "operator": "admin",
            "fileIds": [file_id],
            "dataTableIds": [table_id],
            "workspace": workspace["id"]
        }
    ]
}

headers = {'X-NI-API-KEY': api_key, 'Content-Type': 'application/json'}
create_result_resp = requests.post(f'{sl_uri}/nitestmonitor/v2/results', headers=headers, json=results_info)

# Expect 200, 201, or 204 for success or partial success, raise an exception if there is an error response
create_result_resp.raise_for_status()

result_id = create_result_resp.json()['results'][0]['id']

### Store the result information so that SystemLink can access it
SystemLink uses scrapbook to store result information from each notebook execution to display to the user in the Execution Details slide-out. Here we will display the IDs of the Test Result and DataTable that we created for easy reference later.

In [ ]:
result = [
    {
        'type': 'scalar',
        'id': 'Test Result ID',
        'value': result_id
    },
    {
        'type': 'scalar',
        'id': 'Data Table ID',
        'value': table_id
    }
]

scrapbook.glue('result', result)

# Next Steps

1. Publish this notebook to SystemLink by right-clicking it in the JupyterLab File Browser
1. Manually execute this notebook against a file in the SystemLink Files application
1. Configure a Routine to execute this notebook against any new files that get uploaded to systemlink
1. Use Grafana Dashboards to View and Explore the uploaded data